In [6]:
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from openpyxl import load_workbook

def converter(x):
    return float(x.strip('%'))/100

def convert_percent_to_numeric(x):
    x = x.replace(',', '').replace('%', '')  # Remove commas and percent signs
    x = pd.to_numeric(x, errors='coerce') / 100  # Convert to numeric and divide by 100
    return x

def read_data(data):
    # Read JSON data from the file
    with open(data, 'r') as file:
        data = json.load(file)
    
    # Extract HTML content from the first element in the list (assuming there's only one element)
    html_content = data[0]['body']

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all table rows (tr) within the table
    table_rows = soup.find_all('tr')

    # Extract data into lists
    categories = []
    amounts = []
    negative = False

    for row in table_rows:
        cols = row.find_all('td')
        if len(cols) == 2:  # Assuming each row has exactly 2 columns (label and amount)
            category = cols[0].text.strip()
            amount_str = cols[1].text.strip().replace(',', '')  # Remove commas from numbers
            if amount_str == '-':
                amount_str = np.nan
            elif amount_str:  # Check if amount_str is not empty
                if '(' in amount_str and ')' in amount_str:  # Check if the amount is in parentheses
                    amount_str = amount_str.replace('(', '').replace(')', '')  # Remove parentheses
                    negative = True
                else:
                    negative = False
                
                try:
                    amount = float(amount_str.replace(',', ''))
                except ValueError:
                    amount = None  # Handle cases where conversion fails gracefully
            else:
                amount = None  # Handle empty strings
            
            if negative:
                amount = -amount  # Make the amount negative
            categories.append(category)
            amounts.append(amount)

    # Create a DataFrame
    df = pd.DataFrame({
        'Category': categories,
        'Amount': amounts
    })

    # Display the DataFrame
    # print(df)
    return df

def read_data_ratio(data):
    # Read JSON data from the file
    with open(data, 'r') as file:
        data = json.load(file)
    
    # Extract HTML content from the first element in the list (assuming there's only one element)
    html_content = data[0]['body']

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all table rows (tr) within the table
    table_rows = soup.find_all('tr')

    # Extract data into lists
    categories = []
    herggui = []
    amounts = []

    for row in table_rows:
        cols = row.find_all('td')
        if len(cols) == 3:  # Assuming each row has exactly 3 columns (label, herggui, and amount)
            category = cols[0].text.strip()
            herggui_str = cols[1].text.strip()
            amount_str = cols[2].text.strip().replace(',', '')  # Remove commas from numbers
            if amount_str:  # Check if amount_str is not empty
                try:
                    amount = convert_percent_to_numeric(amount_str)
                except ValueError:
                    amount = None  # Handle cases where conversion fails gracefully
            else:
                amount = None  # Handle empty strings
            categories.append(category)
            herggui.append(herggui_str)
            amounts.append(amount)

    # Create a DataFrame
    df = pd.DataFrame({
        'Category': categories,
        'herggui': herggui,
        'Amount': amounts
    })

    # Display the DataFrame
    # print(df)
    return df


def modify_table_for_regular(df, index):
    df.set_index(df.columns[0], inplace=True)  # Set the second column as the index
    df = df.dropna()

    df.loc["Бусад орлого"] = df.loc["Бусад дэлгэрэнгүй орлого/зардал"]
    indexes_to_keep = [
                "Хүүгийн орлого",
                "Монголбанкинд байршуулсан хөрөнгийн",
                "Банк, санхүүгийн байгууллагад байршуулсан хөрөнгийн",
                "Үнэт цаасны",
                "Зээлийн",
                "Бусад хүүгийн орлого",
                "Хүүгийн зардал",
                "Харилцахад төлсөн хүү",
                "Хадгаламжинд төлсөн хүү",
                "Зээлийн хүүгийн зардал",
                "Үнэт цаасны хүүгийн зардал",
                "Бусад хүүгийн зардал",
                "Хүүгийн цэвэр орлого",
                "Эрсдэлийн сангийн зардал",
                "Эрсдэлийн сангийн дараах цэвэр орлого",
                "Бусад орлого",
                "Хүүгийн бус орлого",
                "Арилжааны орлого",
                "Ханш, үнэлгээний тэгшитгэлийн орлого",
                "Банкны бүтээгдэхүүнтэй холбоотой үйлчилгээний хураамж, шимтгэлийн орлого",
                "Бусад хүүгийн бус орлого",
                "Бусад орлого, олз",
                "Бусад зардал",
                "Хүүгийн бус зардал",
                "Бусад эрсдэлийн сангийн зардал",
                "Арилжааны зардал",
                "Ханш, үнэлгээний тэгшитгэлийн зардал",
                "Хураамж, шимтгэлийн зардал",
                "Үйл ажиллагааны бусад зардал",
                "Бусад зардал, гарз",
                "Татварын өмнөх ашиг, алдагдал",
                "Орлогын татварын зардал",
                "Татварын дараах ашиг, алдагдал",
                "Бусад орлого",
                "Тайлант хугацааны нийт орлогын дүн"
    ]
    df = df.loc[indexes_to_keep]
    return df

def modify_table_for_ratio(df):
    df.set_index(df.columns[0], inplace=True)  # Set the first column as the index
    df = df.drop(labels="herggui", axis=1)
    df = df.drop(["ЗОХИСТОЙ ХАРЬЦААНЫ ШАЛГУУР ҮЗҮҮЛЭЛТ"])
    return df


output_file = "khaan_simple.xlsx"
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    df2 = read_data(f'khaan_df134.json')

    df2 = modify_table_for_regular(df2, 1)


    merged_df = pd.concat([df2], ignore_index=False)
    merged_df.index.names = [1]
    merged_df.rename(columns={merged_df.columns[0]: "ХААН"}, inplace=True)
    merged_df = merged_df.dropna()
    merged_df.to_excel(writer, index=True)


print("successfully!!!!!!!!1")


successfully!!!!!!!!1


/var/folders/6k/jj61jj952yx_x9dsrlst1pz40000gn/T/ipykernel_55817/2322338754.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc["Бусад орлого"] = df.loc["Бусад дэлгэрэнгүй орлого/зардал"]


In [7]:
import pandas as pd
# takenUrl = input("URL link: ")

takenUrl = "https://golomtbank.com/investor-relations/season-report"

tables = pd.read_html(takenUrl)
def convert_third_column_to_numeric(df):
    df[df.columns[0]] = df[df.columns[0]].replace({',': ''}, regex=True)  # Remove commas
    df[df.columns[0]] = pd.to_numeric(df[df.columns[0]], errors='coerce')  # Convert to numeric
    return df

def convert_percent_to_numeric(df):
    df[df.columns[0]] = df[df.columns[0]].replace({',': '', '%': ''}, regex=True)  # Remove commas and percent signs
    df[df.columns[0]] = pd.to_numeric(df[df.columns[0]], errors='coerce') / 100  # Convert to numeric and divide by 100
    return df


def modify_table_for_regular(df, index):
    df.set_index(df.columns[1], inplace=True)
    df = df.drop(labels=0, axis=1)
    df = convert_third_column_to_numeric(df)
    if index == 2:
        print("this is third table")
        # print(df.isnull().sum())
        df.fillna(0, inplace=True)
        # print(df.isnull().sum())
        # print(df.loc[["Арилжааны зардал"], [2]])

        df.loc["Хүүгийн орлого"] = df.loc["Хүүгийн орлого"]
        df.loc["Хүүгийн зардал"] = df.loc["Хүүгийн зардал"]
        df.loc["Эрсдлийн сангийн зардал"] = df.loc["Эрсдэлийн сангийн зардал"]
        df.loc["Монголбанкинд байршуулсан хөрөнгийн"] = df.loc["Монгол банканд байршуулсан хөрөнгийн"]
        df.loc["Сангийн дараах цэвэр хүүгийн орлого"] = df.loc["Эрсдэлийн сангийн дараах цэвэр орлого"]
        
        df.loc["Арилжаа ханшийн тэгштгэлийн ашиг, хэрэгжээгүй олз гарз"] = df.loc["Арилжааны орлого"] - df.loc["Арилжааны зардал"] + df.loc["Ханш, үнэлгээний тэгшитгэлийн орлого"] - df.loc["Ханш, үнэлгээний тэгшитгэлийн зардал"]
        df.loc["Банкны бүтээгдэхүүнтэй холбоотой үйлчилгээний хураамж, шимтгэлийн орлого"] = df.loc["Банкны бүтээгдэхүүнтэй холбоотой үйлчилгээний хураамж, шимтгэлийн орлого"]
        df.loc["Бусад орлого, олз"] = df.loc["Бусад орлого"] - df.loc["Хүүгийн бус орлого"] + df.loc["Бусад хүүгийн бус орлого"]
        df.loc["Үйл ажиллагааны бусад зардал"] = df.loc["Бусад эрсдлийн сангийн зардал"] + df.loc["Хураамж шимтгэлийн зардал"] +df.loc["Үйл ажиллагааны бусад зардал"]
        df.loc["Бусад эрсдэлийн сангийн зардал"] = df.loc["Бусад эрсдлийн сангийн зардал"] 
        df.loc["Бусад 1 зардал"] = df.loc["Бусад зардал, гарз"]
        df.loc["Хураамж, шимтгэлийн зардал"] = df.loc["Хураамж шимтгэлийн зардал"]

        df.loc["Татварын өмнөх ашиг"] = df.loc["Татварын өмнөх ашиг, алдагдал"]
        df.loc["Татвары зардал"] = df.loc["Орлогын татварын зардал"]
        df.loc["Бусад орлого"] = df.loc["Бусад дэлгэрэнгүй орлого"]
        df.loc["Татварын дараах ашиг"] = df.loc["Татварын дараах ашиг, алдагдал"]
        df.loc["Нийт дэлгэрэнгүй орлого"] = df.loc["Тайлант хугацааны нийт орлогын дүн"]

        df.loc["Net profit margin %"] = df.loc["Татварын дараах ашиг"] / (df.loc["Хүүгийн орлого"] + df.loc["Арилжаа ханшийн тэгштгэлийн ашиг, хэрэгжээгүй олз гарз"] + df.loc["Банкны бүтээгдэхүүнтэй холбоотой үйлчилгээний хураамж, шимтгэлийн орлого"] + df.loc["Бусад орлого, олз"])
    
    
        indexes_to_keep = [
                    "Хүүгийн орлого",
                    "Монголбанкинд байршуулсан хөрөнгийн",
                    "Банк, санхүүгийн байгууллагад байршуулсан хөрөнгийн",
                    "Үнэт цаасны",
                    "Зээлийн",
                    "Бусад хүүгийн орлого",
                    "Хүүгийн зардал",
                    "Харилцахад төлсөн хүү",
                    "Хадгаламжинд төлсөн хүү",
                    "Зээлийн хүүгийн зардал",
                    "Үнэт цаасны хүүгийн зардал",
                    "Бусад хүүгийн зардал",
                    "Хүүгийн цэвэр орлого",
                    "Эрсдэлийн сангийн зардал",
                    "Эрсдэлийн сангийн дараах цэвэр орлого", 
                    "Бусад орлого",
                    "Хүүгийн бус орлого",
                    "Арилжааны орлого",
                    "Ханш, үнэлгээний тэгшитгэлийн орлого",
                    "Банкны бүтээгдэхүүнтэй холбоотой үйлчилгээний хураамж, шимтгэлийн орлого",
                    "Бусад хүүгийн бус орлого",
                    "Бусад орлого, олз",
                    "Бусад зардал",
                    "Хүүгийн бус зардал",
                    "Бусад эрсдэлийн сангийн зардал",
                    "Арилжааны зардал",
                    "Ханш, үнэлгээний тэгшитгэлийн зардал",
                    "Хураамж шимтгэлийн зардал",
                    "Үйл ажиллагааны бусад зардал",
                    "Бусад зардал, гарз",
                    "Татварын өмнөх ашиг, алдагдал",
                    "Орлогын татварын зардал",
                    "Татварын дараах ашиг, алдагдал",
                    "Бусад орлого",
                    "Тайлант хугацааны нийт орлогын дүн"
        ]
        df = df.loc[indexes_to_keep]
    return df


# Create a Pandas Excel writer using XlsxWriter as the engine
output_file = "golomt_simple.xlsx"
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    for i in range(0, min(3, len(tables))):
        df = tables[i]
        # print(df)
        if i == 0:
            df1 = modify_table_for_regular(df, i)
            # print(df1)
        elif i == 2:
            df2 = modify_table_for_regular(df, i)
        else:
            continue 
        
    # merged_df = pd.concat([df1, df2, df3], ignore_index=False)
    merged_df = pd.concat([df2], ignore_index=False)
    merged_df.rename(columns={merged_df.columns[0]: "Голомт"}, inplace=True)
    
    # export the merged DataFrame to a different sheets
    merged_df.to_excel(writer, index=True)
print(f"Modified data has been saved to {output_file}")


this is third table
Modified data has been saved to golomt_simple.xlsx


In [8]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

# Disable SSL certificate verification warning (only for development purposes)
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

def get_page_data(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        # Disable SSL verification by setting verify to False
        response = requests.get(url, headers=headers, verify=False)
        response.raise_for_status()  # Check if the request was successful
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching page: {e}")
        return None

def extract_table_data_sda(html):
    try:
        soup = BeautifulSoup(html, 'html.parser')
        tables = soup.find_all('table')
        table_data = []
        for table in tables:
            rows = table.find_all('tr')
            table_rows = []
            for row in rows:
                cols = row.find_all(['th', 'td'])
                table_cols = []
                for col in cols:
                    table_cols.append(col.text.strip())
                table_rows.append(table_cols)
            table_data.append(table_rows)
        print("I WAS HERE XDDDDD")
        return table_data
    except Exception as e:
        print(f"Exception occurred while extracting table data: {str(e)}")
        return None

def display_table_data(url):
    print("I WAS HERE 1")
    html = get_page_data(url)
    # print(html)
    if html:
        table_data = extract_table_data_sda(html)
        print("I WAS HERE 2")
    temp_df = pd.DataFrame(table_data[0][1:], columns=table_data[0][0])
    return temp_df

def read_data(data_file):
    # Read JSON data from the file
    with open(data_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    # Extract HTML content from the first element in the list (assuming there's only one element)
    html_content = data['html']

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all table rows (tr) within the table
    table_rows = soup.find_all('tr')

    # Extract data into lists
    categories = []
    amounts = []

    for row in table_rows:
        cols = row.find_all('td')
        if len(cols) == 3:  # Assuming each row has exactly 2 columns (label and amount)
            category = cols[0].text.strip()
            amount_str = cols[1].text.strip().replace(',', '').replace('(', '').replace(')', '')  # Remove commas from numbers
            if amount_str:  # Check if amount_str is not empty
                try:
                    amount = float(amount_str)
                except ValueError:
                    amount = None  # Handle cases where conversion fails gracefully
            else:
                amount = None  # Handle empty strings
            categories.append(category)
            amounts.append(amount)

    # Create a DataFrame
    df = pd.DataFrame({
        'Category': categories,
        'Amount': amounts
    })

    return df


def convert_percent_to_numeric(df):
    df[df.columns[0]] = df[df.columns[0]].replace({',': '', '%': ''}, regex=True)  # Remove commas and percent signs
    df[df.columns[0]] = pd.to_numeric(df[df.columns[0]], errors='coerce') / 100  # Convert to numeric and divide by 100
    return df



def modify_table_for_regular(df, index):
    df.set_index(df.columns[0], inplace=True)  
    df.fillna(0, inplace=True)
    if index == 1:
        print("this is second table **********************************************************************************************************************")

        df.loc["Эрсдэлийн сангийн зардал"] = df.loc["Эрсдлийн сангийн зардал"]
        df.loc["Эрсдэлийн сангийн дараах цэвэр орлого"] = df.loc["Эрсдлийн сангийн дараах цэвэр орлого"]
        df.loc["Бусад эрсдэлийн сангийн зардал"] = df.loc["Бусад эрсдлийн сангийн зардал"]
        df.loc["Үйл ажиллагааны бусад зардал"] = df.loc["Үйл ажиллагааны зардал"]
        df.loc["Татварын өмнөх ашиг, алдагдал"] = df.loc["Татварын өмнөх ашиг"]
        df.loc["Татварын дараах ашиг, алдагдал "] = df.loc["Тайлант хугацааны ашиг"]
        df.loc["Бусад орлого"] = df.loc["Бусад дэлгэрэнгүй орлого"]
        df.loc["Тайлант хугацааны нийт орлогын дүн"] = df.loc["Нийт дэлгэрэнгүй орлого, татварын дараах байдлаар"]
        indexes_to_keep = [
            "Хүүгийн орлого",
            "Монголбанкинд байршуулсан хөрөнгийн",
            "Банк, санхүүгийн байгууллагад байршуулсан хөрөнгийн",
            "Үнэт цаасны",
            "Зээлийн",
            "Бусад хүүгийн орлого",
            "Хүүгийн зардал",
            "Харилцахад төлсөн хүү",
            "Хадгаламжинд төлсөн хүү",
            "Зээлийн хүүгийн зардал",
            "Үнэт цаасны хүүгийн зардал",
            "Бусад хүүгийн зардал",
            "Хүүгийн цэвэр орлого",
            "Эрсдэлийн сангийн зардал",
            "Эрсдлийн сангийн дараах цэвэр орлого",
            "Бусад орлого",
            "Хүүгийн бус орлого",
            "Арилжааны орлого",
            "Ханш, үнэлгээний тэгшитгэлийн орлого",
            "Банкны бүтээгдэхүүнтэй холбоотой үйлчилгээний хураамж, шимтгэлийн орлого",
            "Бусад хүүгийн бус орлого",
            "Бусад орлого, олз",
            "Бусад зардал",
            "Хүүгийн бус зардал",
            "Бусад эрсдэлийн сангийн зардал",
            "Хураамж, шимтгэлийн зардал",
            "Үйл ажиллагааны бусад зардал",
            "Бусад зардал, гарз",
            "Татварын өмнөх ашиг, алдагдал",
            "Орлогын татварын зардал",
            "Татварын дараах ашиг, алдагдал ",
            "Бусад орлого",
            "Тайлант хугацааны нийт орлогын дүн"
        ]
        df = df.loc[indexes_to_keep]
    
    df = df.dropna()
    df.index.names = [1]
    df.rename(columns={df.columns[0]: "ХАС"}, inplace=True)
    return df

df1 = read_data('xac_df1.json')
df2 = read_data('xac_df2.json')
df3 = display_table_data('https://www.xacbank.mn/page/prudential-ratios')

df2 = modify_table_for_regular(df2, 1)

merged_df = pd.concat([df2 ], ignore_index=False)

merged_df.to_excel("xac_simple.xlsx", index=True)

I WAS HERE 1
I WAS HERE XDDDDD
I WAS HERE 2
this is second table **********************************************************************************************************************


In [9]:
import json
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from openpyxl import load_workbook

def converter(x):
    return float(x.strip('%'))/100

def convert_percent_to_numeric(x):
    x = x.replace(',', '').replace('%', '')  # Remove commas and percent signs
    x = pd.to_numeric(x, errors='coerce') / 100  # Convert to numeric and divide by 100
    return x

def read_data(data):
    # Read JSON data from the file
    with open(data, 'r') as file:
        data = json.load(file)
    
    # Extract HTML content from the first element in the list (assuming there's only one element)
    html_content = data["data"]['content']

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all table rows (tr) within the table
    table_rows = soup.find_all('tr')

    # Extract data into lists
    categories = []
    amounts = []
    negative = False
    for row in table_rows:
        cols = row.find_all('td')

        if len(cols) == 4:  
            category = cols[1].text.strip()
            amount_str = cols[3].text.strip()
        elif len(cols) == 3:  
            category = cols[0].text.strip()
            amount_str = cols[2].text.strip()
        else:
            continue  # Skip rows that don't have 3 or 4 columns

        if amount_str == '-':
            amount = np.nan
        elif amount_str:  # Check if amount_str is not empty
            negative = False
            if '(' in amount_str and ')' in amount_str:  # Check if the amount is in parentheses
                amount_str = amount_str.replace('(', '').replace(')', '')  # Remove parentheses
                negative = True
            
            try:
                amount = float(amount_str.replace(',', ''))
                if negative:
                    amount = -amount  # Make the amount negative
            except ValueError:
                amount = None  # Handle cases where conversion fails gracefully
        else:
            amount = None  # Handle empty strings

        categories.append(category)
        amounts.append(amount)

    # Create a DataFrame
    df = pd.DataFrame({
        'Category': categories,
        'Amount': amounts
    })

    # Display the DataFrame
    return df

def read_data_ratio(data):
    # Read JSON data from the file
    with open(data, 'r') as file:
        data = json.load(file)
    
    # Extract HTML content from the first element in the list (assuming there's only one element)
    html_content = data["data"]['content']

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all table rows (tr) within the table
    table_rows = soup.find_all('tr')

    # Extract data into lists
    categories = []
    herggui = []
    amounts = []

    for row in table_rows:
        cols = row.find_all('td')
        if len(cols) == 3:  # Assuming each row has exactly 3 columns (label, herggui, and amount)
            category = cols[0].text.strip()
            amount_str = cols[2].text.strip().replace(',', '')  # Remove commas from numbers
            if amount_str:  # Check if amount_str is not empty
                try:
                    amount = convert_percent_to_numeric(amount_str)
                except ValueError:
                    amount = None  # Handle cases where conversion fails gracefully
            else:
                amount = None  # Handle empty strings
            categories.append(category)
            amounts.append(amount)

    # Create a DataFrame
    df = pd.DataFrame({
        'Category': categories,
        'Amount': amounts
    })

    # Display the DataFrame
    # print(df)
    return df


def modify_table_for_regular(df):
    df.set_index(df.columns[0], inplace=True)  # Set the second column as the index
    df.fillna(0, inplace=True)
    df = df.dropna()

    print("this is third table")
    # print(df.isnull().sum())
    df.fillna(0, inplace=True)
    # print(df.isnull().sum())
    # print(df.loc[["Арилжааны зардал"], [2]])

    # df.loc["Хүүгийн орлого"] = df.loc["Хүүгийн орлого"]
    # df.loc["Хүүгийн зардал"] = df.loc["Хүүгийн зардал"]
    # df.loc["Эрсдлийн сангийн зардал"] = df.loc["Эрсдэлийн сангийн зардал"]
    # df.loc["Сангийн дараах цэвэр хүүгийн орлого"] = df.loc["Эрсдэлийн сангийн дараах цэвэр орлого"]
    df.loc["Монголбанкинд байршуулсан хөрөнгийн"] = df.loc["Монголбанканд байршуулсан хөрөнгийн"]
    df.loc["Бусад орлого, олз"] = df.loc["Бусад  орлого, олз"]
    df.loc["Бусад орлого"] = df.loc["Бусад дэлгэрэнгүй орлого"]
    
    # df.loc["Арилжаа ханшийн тэгштгэлийн ашиг, хэрэгжээгүй олз гарз"] = df.loc["Арилжааны орлого"] - df.loc["Арилжааны зардал"] + df.loc["Ханш, үнэлгээний тэгшитгэлийн орлого"] - df.loc["Ханш, үнэлгээний тэгшитгэлийн зардал"]
    # df.loc["Банкны бүтээгдэхүүнтэй холбоотой үйлчилгээний хураамж, шимтгэлийн орлого"] = df.loc["Банкны бүтээгдэхүүнтэй холбоотой үйлчилгээний хураамж, шимтгэлийн орлого"]
    # df.loc["Бусад орлого, олз"] = df.loc["Бусад орлого"] - df.loc["Хүүгийн бус орлого"] + df.loc["Бусад хүүгийн бус орлого"]
    # df.loc["Үйл ажиллагааны бусад зардал"] = df.loc["Бусад эрсдлийн сангийн зардал"] + df.loc["Хураамж шимтгэлийн зардал"] +df.loc["Үйл ажиллагааны бусад зардал"]
    # df.loc["Бусад 1 зардал"] = df.loc["Бусад зардал, гарз"]

    # df.loc["Татварын өмнөх ашиг"] = df.loc["Татварын өмнөх ашиг, алдагдал"]
    # df.loc["Татвары зардал"] = df.loc["Орлогын татварын зардал"]
    # df.loc["Татварын дараах ашиг"] = df.loc["Татварын дараах ашиг, алдагдал"]
    # df.loc["Нийт дэлгэрэнгүй орлого"] = df.loc["Тайлант хугацааны нийт орлогын дүн"]

    # df.loc["Net profit margin %"] = df.loc["Татварын дараах ашиг"] / (df.loc["Хүүгийн орлого"] + df.loc["Арилжаа ханшийн тэгштгэлийн ашиг, хэрэгжээгүй олз гарз"] + df.loc["Банкны бүтээгдэхүүнтэй холбоотой үйлчилгээний хураамж, шимтгэлийн орлого"] + df.loc["Бусад орлого, олз"])


    indexes_to_keep = [
                "Хүүгийн орлого",
                "Монголбанкинд байршуулсан хөрөнгийн",
                "Банк, санхүүгийн байгууллагад байршуулсан хөрөнгийн",
                "Үнэт цаасны",
                "Зээлийн",
                "Бусад хүүгийн орлого",
                "Хүүгийн зардал",
                "Харилцахад төлсөн хүү",
                "Хадгаламжинд төлсөн хүү",
                "Зээлийн хүүгийн зардал",
                "Үнэт цаасны хүүгийн зардал",
                "Бусад хүүгийн зардал",
                "Хүүгийн цэвэр орлого",
                "Эрсдэлийн сангийн зардал",
                "Эрсдэлийн сангийн дараах цэвэр орлого",
                "Бусад орлого",
                "Хүүгийн бус орлого",
                "Арилжааны орлого",
                "Ханш, үнэлгээний тэгшитгэлийн орлого",
                "Банкны бүтээгдэхүүнтэй холбоотой үйлчилгээний хураамж, шимтгэлийн орлого",
                "Бусад хүүгийн бус орлого",
                "Бусад орлого, олз",
                "Бусад зардал",
                "Хүүгийн бус зардал",
                "Бусад эрсдэлийн сангийн зардал",
                "Арилжааны зардал",
                "Ханш, үнэлгээний тэгшитгэлийн зардал",
                "Хураамж, шимтгэлийн зардал",
                "Үйл ажиллагааны бусад зардал",
                "Бусад зардал, гарз",
                "Татварын өмнөх ашиг, алдагдал",
                "Орлогын татварын зардал",
                "Татварын дараах ашиг, алдагдал",
                "Бусад орлого",
                "Тайлант хугацааны нийт орлогын дүн"
    ]
    df = df.loc[indexes_to_keep]

    return df

def modify_table_for_ratio(df):
    print("this is third table **********************************************************************************************************************")
    df.set_index(df.columns[0], inplace=True)  # Set the first column as the index
    return df


output_file = "state_simple.xlsx"
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    df1_2 = read_data(f'state.json')


    df1_2 = modify_table_for_regular(df1_2)

    merged_df = pd.concat([df1_2], ignore_index=False)
    # print(merged_df)
    merged_df.index.names = [1]
    merged_df.rename(columns={merged_df.columns[0]: "Төрийн банк"}, inplace=True)
    merged_df = merged_df.dropna()
    merged_df.to_excel(writer, index=True)


print("successfully!!!!!!!!1")


this is third table
successfully!!!!!!!!1


In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

takenURL = 'https://transbank.mn/about-us/financial-information/quarterly-financial-statements'
response = requests.get(takenURL)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    tables = soup.find_all('table')
    
    df = []
    for idx, table in enumerate(tables):
        if idx == 2:  # Assuming the target table is the third table (index 2)
            rows = []
            for row in table.find_all('tr'):
                columns = row.find_all(['th', 'td'])
                row_data = [column.text.strip() for column in columns]
                if (idx in [0, 2] and len(row_data) == 3) or (i in [1, 3] and len(row_data) in [2, 3]):
                    rows.append(row_data)
            df2 = pd.DataFrame(rows)
            break


def convert_numeric(df):
    df[df.columns[0]] = df[df.columns[0]].replace({',': ''}, regex=True)  # Remove commas
    df[df.columns[0]] = pd.to_numeric(df[df.columns[0]], errors='coerce')  # Convert to numeric
    return df
def convert_percent_to_numeric(df):
    df[df.columns[0]] = df[df.columns[0]].replace({',': '', '%': ''}, regex=True)  # Remove commas and percent signs
    df[df.columns[0]] = pd.to_numeric(df[df.columns[0]], errors='coerce') / 100  # Convert to numeric and divide by 100
    return df


def modify_table_for_regular(df, index):
    df = df.drop([df.columns[0]], axis=1, inplace=False)
    df = df.drop([14, 16, 15])
    df.set_index(df.columns[0], inplace=True)  # Set column as the index
    df = convert_numeric(df)
    df.to_excel("psdave.xlsx", index=True)
    if index == 1:
        print("this is second table **********************************************************************************************************************")
        df.fillna(0, inplace=True)
        # print(df.loc[["Арилжааны зардал"], [2]])

        df.loc["Хүүгийн орлого"] = df.loc["Хүүний орлого"]
        df.loc["Хүүгийн зардал"] = df.loc["Хүүний зардал"]
        df.loc["Хүүгийн бус орлого"] = df.loc["Хүүний бус орлого"]
        df.loc["Хүүгийн бус зардал"] = df.loc["Хүүний бус зардал"]
        df.loc["Монголбанкинд байршуулсан хөрөнгийн"] = df.loc["Монголбанканд байршуулсан хөрөнгийн"]
        df.loc["Бусад хүүгийн орлого"] = df.loc["Бусад хүүний орлого"]
        df.loc["Бусад хүүгийн зардал"] = df.loc["Бусад хүүний зардал"]
        df.loc["Хадгаламжинд төлсөн хүү"] = df.loc["Хадгаламжид төлсөн хүү"]
        df.loc["Зээлийн хүүгийн зардал"] = df.loc["Зээлийн хүүний зардал"]
        df.loc["Үнэт цаасны хүүгийн зардал"] = df.loc["Үнэт цаасны хүүний зардал"]
        df.loc["Эрсдлийн сангийн зардал"] = df.loc["Эрсдэлийн сангийн зардал"]


        df.loc["Хүүгийн цэвэр орлого"] = df.loc["Цэвэр хүүний орлого (1-2)"]
        df.loc["Эрсдэлийн сангийн дараах цэвэр орлого"] = df.loc["Эрсдэлийн сангийн дараах цэвэр орлого (3-4)"]

        
        df.loc["Банкны бүтээгдэхүүнтэй холбоотой үйлчилгээний хураамж, шимтгэлийн орлого"] = df.loc["Банкны бүтээгдэхүүнтэй холбоотой үйлчилгээний хураамж, шимтгэлийн орлого"] 
        
        df.loc["Бусад орлого, олз"] = df.loc["Бусад  орлого, олз"]
        df.loc["Бусад хүүгийн бус орлого"] = df.loc["Бусад хүүний бус орлого"]

        df.loc["Татварын өмнөх ашиг, алдагдал"] = df.loc["Татварын өмнөх ашиг (5+6-7)"]
        df.loc["Орлогын татварын зардал"] = df.loc["Орлогын албан татварын зардал"]
        df.loc["Татварын дараах ашиг, алдагдал"] = df.loc["Татварын дараах ашиг (8-9)"]
        df.loc["Бусад орлого"] = df.loc["Бусад дэлгэрэнгүй орлого"]
        indexes_to_keep = [
            "Хүүгийн орлого","Монголбанкинд байршуулсан хөрөнгийн", "Банк, санхүүгийн байгууллагад байршуулсан хөрөнгийн", "Үнэт цаасны",
            "Зээлийн", "Бусад хүүгийн орлого", "Хүүгийн зардал", "Харилцахад төлсөн хүү", "Хадгаламжинд төлсөн хүү",
            "Зээлийн хүүгийн зардал", "Үнэт цаасны хүүгийн зардал", "Бусад хүүгийн зардал", "Хүүгийн цэвэр орлого", "Эрсдэлийн сангийн зардал",
            "Эрсдэлийн сангийн дараах цэвэр орлого", "Бусад орлого", "Хүүгийн бус орлого", "Арилжааны орлого",
            "Ханш, үнэлгээний тэгшитгэлийн орлого",
            "Банкны бүтээгдэхүүнтэй холбоотой үйлчилгээний хураамж, шимтгэлийн орлого", "Бусад хүүгийн бус орлого", "Бусад орлого, олз", "Бусад зардал", "Хүүний бус зардал",
            "Бусад эрсдэлийн сангийн зардал", "Арилжааны зардал","Ханш, үнэлгээний тэгшитгэлийн зардал", "Хураамж, шимтгэлийн зардал",
            "Үйл ажиллагааны бусад зардал", "Бусад зардал, гарз", "Татварын өмнөх ашиг, алдагдал", "Орлогын татварын зардал",
            "Татварын дараах ашиг, алдагдал","Бусад орлого", "Тайлант хугацааны нийт дэлгэрэнгүй орлогын дүн"
        ]
        df = df.loc[indexes_to_keep]
    
    df = df.dropna()
    df.rename(columns={df.columns[0]: 2}, inplace=True)
    return df



df2 = modify_table_for_regular(df2, 1)
merged_df = pd.concat([df2], ignore_index=False)

merged_df.rename(columns={merged_df.columns[0]: "Транс"}, inplace=True)


merged_df.to_excel("trans_simple.xlsx", index=True)
# print(merged_df)







this is second table **********************************************************************************************************************


In [11]:
import pandas as pd

khaan = pd.read_excel('khaan_simple.xlsx')
golomt = pd.read_excel('golomt_simple.xlsx')
state = pd.read_excel('state_simple.xlsx')
trans = pd.read_excel('trans_simple.xlsx')
xac = pd.read_excel('xac_simple.xlsx')

# Reset index to numeric index to avoid index alignment issues
khaan.reset_index(drop=True, inplace=True)
golomt.reset_index(drop=True, inplace=True)
state.reset_index(drop=True, inplace=True)
trans.reset_index(drop=True, inplace=True)
xac.reset_index(drop=True, inplace=True)

# Merge ignoring the index
merged_df = pd.concat([khaan, golomt, state, trans, xac], axis=1)

# Save to Excel
merged_df.to_excel('all_bank_simple.xlsx', index=False)
